Ноутбук для примера работы с сервисом.

Датасет: https://www.kaggle.com/datasets/mirichoi0218/insurance
Таргет - стоимость услуг по ДМС 

Перед запуском ячеек убедитесь, что gRPC сервер запущен (см инструкцию по запуску в README)

In [40]:
import sys
import os
import json
import numpy as np
import pandas as pd
import grpc
import urllib.request
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

sys.path.insert(0, os.path.dirname(os.path.dirname(os.path.abspath(''))))

from grpc_client import service_pb2, service_pb2_grpc

In [41]:
import kagglehub
import shutil

path = kagglehub.dataset_download("mirichoi0218/insurance")

try:
    kaggle_dir = os.path.join(path, "insurance.csv")
    if os.path.exists("insurance.csv"):
        df = pd.read_csv("insurance.csv")
    elif os.path.exists(kaggle_dir):
        df = pd.read_csv(kaggle_dir)
        try:
            import shutil; shutil.copy(kaggle_dir, "insurance.csv")
        except Exception:
            pass
except Exception as e:
    print("Файл не загружен, загрузите вручную или попробуйте позже")
    raise


In [42]:
# категориальные
le_sex = LabelEncoder()
le_smoker = LabelEncoder()
le_region = LabelEncoder()

df_processed = df.copy()
df_processed['sex'] = le_sex.fit_transform(df['sex'])
df_processed['smoker'] = le_smoker.fit_transform(df['smoker'])
df_processed['region'] = le_region.fit_transform(df['region'])

X = df_processed[['age', 'sex', 'bmi', 'children', 'smoker', 'region']].values
y = df_processed['charges'].values

split_idx = int(len(X) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

print(f"\nTrain: {X_train.shape[0]}")
print(f"Test: {X_test.shape[0]}")



Train: 1070
Test: 268


In [43]:
# Инициализируем gRPC клиент
channel = grpc.insecure_channel('localhost:50051')
stub = service_pb2_grpc.MLModelServiceStub(channel)


In [44]:
# проверяем сервис
request = service_pb2.HealthCheckRequest()
response = stub.HealthCheck(request)
print(f"Статус: {response.status}")
print(f"Ответ: {response.message}")


Статус: healthy
Ответ: Service is running


In [45]:
# для инфо показываем какие есть классы моделей

request = service_pb2.ListModelClassesRequest()
response = stub.ListModelClasses(request)

print(f"Доступные классы моделей: {len(response.available_models)}")
for model in response.available_models:
    print(f"\n- {model.model_class}")
    print(f"  Гиперпараметры:")
    for param_name, param_info in model.hyperparameter_schema.items():
        default_val = param_info.default_value if param_info.default_value else "None"
        print(f"    - {param_name}: {param_info.type} (default: {default_val})")


Доступные классы моделей: 2

- LinearRegression
  Гиперпараметры:
    - fit_intercept: bool (default: true)
    - n_jobs: int (default: None)

- RandomForest
  Гиперпараметры:
    - max_depth: int (default: None)
    - random_state: int (default: None)
    - max_features: str (default: "sqrt")
    - min_samples_leaf: int (default: 1)
    - n_estimators: int (default: 100)
    - min_samples_split: int (default: 2)


In [46]:
data_points = [service_pb2.DataPoint(values=row.tolist()) for row in X_train]

hyperparameters = {
    "fit_intercept": "true",
    "n_jobs": "-1"
}

# Создаем запрос на обучение модели
request = service_pb2.TrainModelRequest(
    model_class="LinearRegression",
    hyperparameters=hyperparameters,
    X=data_points,
    y=y_train.tolist()
)

# Обучаем
response = stub.TrainModel(request)

if response.model_id:
    print(f"Модель обучена\n ID: {response.model_id}\n Класс: {response.model_class}\n Ответ: {response.message}")
    model_id = response.model_id
else:
    print("Что-то пошло не так")
    raise Exception("Модель не обучена")


Модель обучена
 ID: 4f787827-8390-4b7a-a4b6-c484238bbb78
 Класс: LinearRegression
 Ответ: Model trained successfully


In [47]:
data_points = [service_pb2.DataPoint(values=row.tolist()) for row in X_test]

# Создаем запрос на предсказание модели
request = service_pb2.PredictRequest(
    model_id=model_id,
    X=data_points
)

# Получаем его
response = stub.Predict(request)
y_pred = np.array(response.predictions)

print(f"Предсказание получено")

Предсказание получено


In [48]:
# Можем посмотреть метрики ради интереса
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mape = abs(np.mean((y_test - y_pred) / y_test)) * 100
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"MAPE: {mape:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R^2: {r2:.4f}")

MAE: 4232.10
MSE: 37239419.49
MAPE: 12.62
RMSE: 6102.41
R^2: 0.7581


In [49]:
# Выводим все обученные модели в реестре
request = service_pb2.ListModelsRequest()
response = stub.ListModels(request)

print(f"Всего моделей обучено: {len(response.models)}")
for model in response.models:
    print(f"\n- ID: {model.model_id}")
    print(f"  Класс: {model.model_class_name}")
    print(f"  Создана: {model.created_at}")
    print(f"  Обучена: {model.is_trained}")
    if model.hyperparameters:
        print(f"  Гиперпараметры: {dict(model.hyperparameters)}")


Всего моделей обучено: 3

- ID: b5167230-640b-4ee0-acf8-71db50e5b6a1
  Класс: LinearRegression
  Создана: 2025-11-09T20:53:24.985981
  Обучена: True
  Гиперпараметры: {'fit_intercept': 'true', 'n_jobs': '-1'}

- ID: 03180824-79af-48c7-b114-029baa2bbbb6
  Класс: LinearRegression
  Создана: 2025-11-09T21:20:15.170781
  Обучена: True
  Гиперпараметры: {'fit_intercept': 'true', 'n_jobs': '-1'}

- ID: 4f787827-8390-4b7a-a4b6-c484238bbb78
  Класс: LinearRegression
  Создана: 2025-11-09T21:35:32.101928
  Обучена: True
  Гиперпараметры: {'fit_intercept': 'true', 'n_jobs': '-1'}


In [ ]:
# Если хотим удалить модель и закончить
request = service_pb2.DeleteModelRequest(model_id=model_id)
response = stub.DeleteModel(request)


channel.close()